In [ ]:
import os
import pandas as pd

#####################################################
# Write the Big5 scores obtained from the Qualistics CSV into the corresponding ID's CSV files from Pavlovia
#####################################################

# Big 5スコアが記載されているCSVファイルのパス
big5_csv_path = r'C:\Users\kawamura.n\Desktop\sentence_simirality\data\Analysis_by_trait\反転項目のスコアを反転して5特性の平均スコアを出した.csv'

# 類似性評価CSVファイルが入っているフォルダのパス
folder_path = r'C:\Users\kawamura.n\Desktop\sentence_simirality\data\Analysis_by_trait\data_Pavlovia'

# 処理後のCSVファイルを保存する新しいフォルダのパス
output_folder = r'C:\Users\kawamura.n\Desktop\New folder'

# 新しいフォルダが存在しない場合は作成する
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Big 5スコアのCSVファイルを読み込む
big5_df = pd.read_csv(big5_csv_path)

# Big 5スコアの列名をわかりやすい名前に変更する（必要に応じて）
big5_columns = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']
big5_df.columns = ['LocationLatitude', 'LocationLongitude', 'Q1', 'Q5', 'Q10_1', 'Q10_2',
                   'Q10_3', 'Q10_4', 'Q10_5', 'Q10_6', 'Q10_7', 'Q10_8', 'Q10_9', 'Q10_10',
                   'Q10_11', 'Q10', 'Extraversion', 'Agreeableness', 'Conscientiousness', 
                   'Neuroticism', 'Openness']

# フォルダ内のすべてのCSVファイルに対して処理を行う
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # 類似性評価のCSVファイルを読み込む
        file_path = os.path.join(folder_path, filename)
        similarity_df = pd.read_csv(file_path)

        # CSVに含まれるIDを取得し、それに対応するBig 5スコアを見つける
        prolific_id = similarity_df['Prolific_ID*'].iloc[0]
        individual_big5_scores = big5_df[big5_df['Q1'] == prolific_id]

        if not individual_big5_scores.empty:
            # Big 5スコアを類似性評価CSVに統合
            for trait in ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']:
                similarity_df[trait] = individual_big5_scores[trait].values[0]

            # 新しいフォルダに保存（ファイル名を保持したまま新しいフォルダに保存）
            output_file_path = os.path.join(output_folder, filename)
            similarity_df.to_csv(output_file_path, index=False)
        else:
            print(f"{prolific_id}に対応するBig 5スコアが見つかりませんでした。")

print("すべてのファイルにBig 5スコアを統合し、新しいフォルダに保存しました。")




#####################################################
# Convert the simirality ratings to dissimirality ratings for all CSV files inside the folder
#####################################################

# 変換ルールの定義
similarity_conversion_rule = {
    4: 0, 3: 1, 2: 2, 1: 3,
    -1: 4, -2: 5, -3: 6, -4: 7
}

# 変換後のCSVファイルを保存する新しいフォルダのパス
converted_folder = input("Please enter the output folder path: ")

# フォルダ内の全てのcsvファイルに対して処理を行う
for file_name in os.listdir(output_folder):
    if file_name.endswith('.csv'):
        # ファイルのフルパスを取得
        input_file_path = os.path.join(output_folder, file_name)
        
        # CSVファイルを読み込む
        df = pd.read_csv(input_file_path)
        
        # "similarity"列が存在する場合のみ変換を行う
        if 'similarity' in df.columns:
            df['similarity'] = df['similarity'].map(lambda x: similarity_conversion_rule.get(x, x))
        
        # 新しいフォルダにファイルを保存
        output_file_path = os.path.join(converted_folder, file_name)
        df.to_csv(output_file_path, index=False)
        
        print(f'変換完了: {file_name}')





#####################################################
# List all sentence pairs used to analysis along with dessimirality ratings and big5 scores of the individuals who provided those ratings
#####################################################


# フォルダのパスを指定
folder_path = converted_folder

# Big Fiveの列名を指定
big5_columns = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
similarity_column = 'similarity'
stimulus_pair_columns = ['displayed_text_above', 'displayed_text_below']

# 結果を格納するためのリスト
output_data = []

# フォルダ内のすべてのcsvファイルに対して処理を行う
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # ファイルのフルパスを取得
        file_path = os.path.join(folder_path, file_name)
        
        # CSVファイルを読み込む
        df = pd.read_csv(file_path)
        
        # 14行目から88行目のデータを抽出
        df_subset = df.iloc[13:88]

        # 必要な列を抽出
        for index, row in df_subset.iterrows():
            result = {
                'file': file_name,
                'stimulus1': row[stimulus_pair_columns[0]],
                'stimulus2': row[stimulus_pair_columns[1]],
                'similarity': row[similarity_column]
            }
            # Big Five特性の値も追加
            for trait in big5_columns:
                result[trait] = row[trait]
            
            output_data.append(result)

# 結果をデータフレームに変換して保存
output_df = pd.DataFrame(output_data)
output_df.to_csv('combined_stimulus_similarity_big5.csv', index=False)
